In [4]:
import os
import datetime
import argparse
from rich.console import Console
from rich.tree import Tree
from rich.text import Text
from rich import box

# 定义常见的代码文件扩展名
CODE_EXTENSIONS = {
    '.py', '.js', '.java', '.cpp', '.c', '.cs', '.rb', '.go',
    '.ts', '.php', '.html', '.css', '.swift', '.kt', '.rs', '.scala',
    '.pl', '.sh', '.bat', '.ps1', '.txt', '.toml', '.cfg', '.h'
}

# 扩展名到语言名称的映射（用于语法高亮，可根据需要扩展）
EXTENSION_LANGUAGE_MAP = {
    '.py': 'python',
    '.js': 'javascript',
    '.java': 'java',
    '.cpp': 'cpp',
    '.c': 'c',
    '.cs': 'csharp',
    '.rb': 'ruby',
    '.go': 'go',
    '.ts': 'typescript',
    '.php': 'php',
    '.html': 'html',
    '.css': 'css',
    '.swift': 'swift',
    '.kt': 'kotlin',
    '.rs': 'rust',
    '.scala': 'scala',
    '.pl': 'perl',
    '.sh': 'bash',
    '.bat': 'dos',
    '.ps1': 'powershell',
    '.txt': 'text',      # 对于纯文本文件
    '.toml': 'toml',
    '.cfg': 'ini',        # 使用 ini 作为配置文件的语法高亮
    '.h': 'c',            # 添加 .h 文件的语法高亮（视具体需求调整为 'cpp'）
}

# 定义图标
DIR_ICON = "📁"
FILE_ICON = "📄"
ERROR_ICON = "❌"
WARNING_ICON = "⚠️"

console = Console()

def probe_directory_plain(path, plain_text_lines, only_dirs=False, show_code_content=False, prefix=""):
    """
    递归探测目录结构并构建纯文本目录树
    :param path: 要探测的目录路径
    :param plain_text_lines: 用于收集纯文本行的列表
    :param only_dirs: 是否仅显示目录结构
    :param show_code_content: 是否显示代码文件的内容
    :param prefix: 用于纯文本树的前缀
    """
    try:
        # 获取目录内容
        items = os.listdir(path)
    except PermissionError:
        plain_text_lines.append(f"{prefix}{WARNING_ICON} 没有权限访问: {path}")
        return
    except Exception as e:
        plain_text_lines.append(f"{prefix}{ERROR_ICON} 错误: {e}")
        return

    items_sorted = sorted(items)
    for index, item in enumerate(items_sorted):
        full_path = os.path.join(path, item)
        is_last = index == len(items_sorted) - 1
        connector = "└── " if is_last else "├── "
        new_prefix = "    " if is_last else "│   "

        try:
            stats = os.stat(full_path)
            size = stats.st_size
            modified_time = datetime.datetime.fromtimestamp(stats.st_mtime)
        except Exception as e:
            plain_text_lines.append(f"{prefix}{connector}{ERROR_ICON} 无法获取文件信息: {item} - {e}")
            continue

        if os.path.isdir(full_path):
            # 目录
            plain_text_lines.append(f"{prefix}{connector}{DIR_ICON} {item}/")
            probe_directory_plain(
                full_path,
                plain_text_lines,
                only_dirs=only_dirs,
                show_code_content=show_code_content,
                prefix=prefix + new_prefix
            )
        else:
            if only_dirs:
                continue
            _, ext = os.path.splitext(item)
            if ext.lower() in CODE_EXTENSIONS:
                line = f"{prefix}{connector}{FILE_ICON} {item}"
                plain_text_lines.append(line)
                if show_code_content:
                    try:
                        with open(full_path, 'r', encoding='utf-8') as f:
                            code_content = f.read()
                        indented_code = "\n".join([prefix + new_prefix + line for line in code_content.splitlines()])
                        plain_text_lines.append(indented_code)
                    except Exception as e:
                        plain_text_lines.append(f"{prefix}{new_prefix}{ERROR_ICON} 无法读取文件内容: {e}")
            else:
                # 如果需要显示所有文件，可以取消注释下行
                # plain_text_lines.append(f"{prefix}{connector}{FILE_ICON} {item}")
                pass

def probe_directory_rich(path, tree, only_dirs=False, show_code_content=False):
    """
    递归探测目录结构并构建 Rich 的 Tree 对象
    :param path: 要探测的目录路径
    :param tree: Rich 的 Tree 对象
    :param only_dirs: 是否仅显示目录结构
    :param show_code_content: 是否显示代码文件的内容
    """
    try:
        items = sorted(os.listdir(path))
    except PermissionError:
        tree.add(f"{WARNING_ICON} 没有权限访问: {path}")
        return
    except Exception as e:
        tree.add(f"{ERROR_ICON} 错误: {e}")
        return

    for index, item in enumerate(items):
        full_path = os.path.join(path, item)
        try:
            stats = os.stat(full_path)
            size = stats.st_size
            modified_time = datetime.datetime.fromtimestamp(stats.st_mtime)
        except Exception as e:
            tree.add(f"{ERROR_ICON} 无法获取文件信息: {item} - {e}")
            continue

        if os.path.isdir(full_path):
            # 目录
            branch = tree.add(f"{DIR_ICON} [bold]{item}/[/bold]")
            probe_directory_rich(
                full_path,
                branch,
                only_dirs=only_dirs,
                show_code_content=show_code_content
            )
        else:
            if only_dirs:
                continue
            _, ext = os.path.splitext(item)
            if ext.lower() in CODE_EXTENSIONS:
                if show_code_content:
                    try:
                        with open(full_path, 'r', encoding='utf-8') as f:
                            code_content = f.read()
                        branch = tree.add(f"{FILE_ICON} [italic]{item}[/italic]")
                        tree_code = branch.add(f"[gray70]{code_content}[/gray70]")
                    except Exception as e:
                        tree.add(f"{ERROR_ICON} 无法读取文件内容: {item} - {e}")
                else:
                    tree.add(f"{FILE_ICON} [italic]{item}[/italic]")
            else:
                # 如果需要显示所有文件，可以取消注释下行
                # tree.add(f"{FILE_ICON} {item}")
                pass

def display_directory_tree_rich(parsed_args):
    """
    使用 Rich 库在控制台中显示目录结构
    :param parsed_args: 解析后的命令行参数
    """
    target_dir = parsed_args.target_dir or os.getcwd()
    only_dirs = parsed_args.only_dirs
    show_code = parsed_args.show_code

    tree = Tree(f"{DIR_ICON} [bold]{os.path.abspath(target_dir)}/[/bold]", guide_style="bold bright_blue")
    probe_directory_rich(target_dir, tree, only_dirs=only_dirs, show_code_content=show_code)
    console.print(tree)

def display_directory_tree_plain(parsed_args):
    """
    将目录结构写入到纯文本文件中
    :param parsed_args: 解析后的命令行参数
    """
    target_dir = parsed_args.target_dir or os.getcwd()
    only_dirs = parsed_args.only_dirs
    show_code = parsed_args.show_code
    output_file = parsed_args.output_file

    plain_text_lines = [f"{DIR_ICON} {os.path.abspath(target_dir)}/"]
    probe_directory_plain(
        target_dir,
        plain_text_lines,
        only_dirs=only_dirs,
        show_code_content=show_code
    )
    try:
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write("\n".join(plain_text_lines))
        console.print(f"{FILE_ICON} 目录结构已成功写入到文件：{output_file}")
    except Exception as e:
        console.print(f"{ERROR_ICON} 无法写入文件 {output_file}: {e}")

def display_directory_tree(target_dir=None, only_dirs=False, show_code_content=False, output_file=None):
    """
    显示目录结构并可选地将输出写入到一个 txt 文件
    :param target_dir: 要探测的目录路径。如果为 None，则使用当前工作目录
    :param only_dirs: 是否仅显示目录结构
    :param show_code_content: 是否显示代码文件的内容
    :param output_file: 输出的 txt 文件路径。如果为 None，则在控制台中以富文本显示
    """
    if output_file is not None:
        # 输出到 txt 文件
        display_directory_tree_plain(
            parsed_args=argparse.Namespace(
                target_dir=target_dir,
                only_dirs=only_dirs,
                show_code=show_code_content,
                output_file=output_file
            )
        )
    else:
        # 在控制台中以 Rich 格式显示
        display_directory_tree_rich(
            parsed_args=argparse.Namespace(
                target_dir=target_dir,
                only_dirs=only_dirs,
                show_code=show_code_content,
                output_file=None
            )
        )

display_directory_tree(
    target_dir='C:\\Users\\Inuyasha\\Programs\\AI_Native_WorkFlow\\code_base\\example_program\\medical_resource_optimization\\medical_opt',
    only_dirs=False,
    show_code_content=True,
    output_file=r'C:\Users\Inuyasha\Programs\AI_Native_WorkFlow\code_base\example_program\medical_resource_optimization\medical_opt\output.txt'
)


📄 
目录结构已成功写入到文件：C:\Users\Inuyasha\Programs\AI_Native_WorkFlow\code_base\example_program\medical_resource_
optimization\medical_opt\output.txt

In [ ]:
import os
import re

def create_structure(input_str):
    """
    解析给定的目录结构字符串，并在对应的位置创建文件夹和文件。

    参数:
    input_str (str): 包含目录和文件结构的多行字符串。
    """
    lines = input_str.strip().splitlines()
    if not lines:
        print("输入字符串为空。")
        return
    
    # 解析根路径
    base_path_line = lines[0].strip()
    # 修正正则表达式，添加缺失的方括号和引号
    base_path_match = re.match(r'^📁\s+(.+)[/\\]$', base_path_line)
    if not base_path_match:
        print("根路径格式不正确。")
        print(f"尝试匹配的根路径行: {base_path_line}")
        return
    
    base_path = base_path_match.group(1)
    base_path = os.path.normpath(base_path)  # 标准化路径格式
    print(f"根路径: {base_path}")
    
    # 确保根路径存在
    os.makedirs(base_path, exist_ok=True)
    
    # 用于跟踪当前路径的栈
    path_stack = []
    
    # 遍历每一行，解析并创建对应的文件夹或文件
    for line in lines[1:]:
        # 使用正则表达式匹配缩进、连接符和名称
        match = re.match(r'^((?:┃\s{3}|    )*)(┣━━ |┗━━ )(?:📄 |📁 )?(.*)', line)
        if not match:
            print(f"无法解析的行: {line}")
            continue  # 跳过无法解析的行
        
        indent_str, connector, name = match.groups()
        
        # 计算当前的缩进层级，每个 '┃   ' 或 '    ' 代表一个层级
        indent_units = indent_str.count('┃   ') + indent_str.count('    ')
        
        # 调整路径栈到当前层级
        path_stack = path_stack[:indent_units]
        
        # 添加当前目录或文件到路径栈
        path_stack.append(name)
        
        # 构建完整的路径
        current_path = os.path.join(base_path, *path_stack)
        
        # 根据行中的符号决定是创建文件夹还是文件
        if '📁' in line:
            # 创建目录
            try:
                os.makedirs(current_path, exist_ok=True)
                print(f"创建文件夹: {current_path}")
            except Exception as e:
                print(f"创建文件夹失败 {current_path}: {e}")
        elif '📄' in line:
            # 确保父目录存在
            parent_dir = os.path.dirname(current_path)
            os.makedirs(parent_dir, exist_ok=True)
            try:
                # 创建空文件
                with open(current_path, 'a', encoding='utf-8'):
                    os.utime(current_path, None)
                print(f"创建文件: {current_path}")
            except Exception as e:
                print(f"创建文件失败 {current_path}: {e}")
        else:
            print(f"未知的类型 (未标记 📁 或 📄): {line}")

# 示例输入
input_structure = """
📁 C:\\Users\\Inuyasha\\Programs\\AI_Native_WorkFlow\\code_base\\example_program\\medical_resource_optimization/
┣━━ 📁 documents/
┃   ┣━━ 📄 problem_description.md
┃   ┗━━ 📄 mathematical_model.md
┣━━ 📁 test_medical_opt/
┃   ┣━━ 📄 test_01_data_loader.py
┃   ┣━━ 📄 test_02_ahp.py
┃   ┣━━ 📄 test_03_fuzzy.py
┃   ┣━━ 📄 test_04_objective.py
┃   ┣━━ 📄 test_05_constraints.py
┃   ┣━━ 📄 test_06_optimizer.py
┃   ┣━━ 📄 test_07_visualizer.py
┃   ┗━━ 📄 test_08_integration.py
┣━━ 📁 medical_opt/
┃   ┣━━ 📄 __init__.py
┃   ┣━━ 📄 p01_data_loader.py
┃   ┣━━ 📄 p02_ahp.py
┃   ┣━━ 📄 p03_fuzzy.py
┃   ┣━━ 📄 p04_objective.py
┃   ┣━━ 📄 p05_constraints.py
┃   ┣━━ 📄 p06_optimizer.py
┃   ┣━━ 📄 p07_visualizer.py
┃   ┣━━ 📄 p08_utils.py
┃   ┗━━ 📄 config.py
┣━━ 📄 main.py
┣━━ 📄 requirements.txt
┗━━ 📄 README.md

"""

# 调用函数创建结构
create_structure(input_structure)
